### 이미지 데이터 분류
- 데이터\t: sklearn.datasets의 MNIST => 디지털 숫자 0 ~ 9 손글씨 데이터셋
- 피쳐\t: 이미지 픽셀 수, 28x28 = 784개
- 타겟\t: 0 ~ 9, 총 10개
- 학습유형\t: 지도학습 + 분류

(1) 모듈 로딩 및 데이터 준비

In [1]:
# 모듈 로딩
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml('mnist_784')   # sklearn.utils._bunch.Bunch

C:\Users\kdp\.conda\envs\EXAM_ML\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

In [11]:
type(mnist.data), type(mnist.target)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [13]:
# mnist 데이터 저장
# mnist.data.to_csv('mnist.data.csv', index=False)
# mnist.target.to_csv('mnist.target.csv', index=False)

In [7]:
mnist.data.describe()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,...,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.0,70000.0,70000.0,70000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.197414,0.099543,0.046629,0.016614,0.012957,0.001714,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.991206,4.256304,2.783732,1.561822,1.553796,0.320889,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.000000,0.0,0.0,0.0,0.0


(2) 데이터 전처리 <hr>
- 이미지 픽셀값 0~1로 스케일링 

In [8]:
featureDF = mnist.data

In [24]:
featureDF = featureDF/255  # 스케일링 (MinMaxScaler 써도 된다.)

In [25]:
targetSR = mnist.target

In [26]:
targetSR.unique()

['5', '0', '4', '1', '9', '2', '3', '6', '7', '8']
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

(3) 기계학습용 데이터셋 준비 <hr>
- 학습용, 검증용, 테스트용 3개 데이터셋 준비

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
# 학습용 / 테스트용 분리
x_train, x_test, y_train, y_test = featureDF.iloc[:60000], featureDF.iloc[60000:], targetSR[:60000], targetSR[60000:]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [29]:
# 학습용 & 검증용 분리 => 학습용 데이터셋에서 일부를 검증용으로 준비
# 검증용 : 학습 시 모델 최적화가 잘 되는지 검사하는 데이터셋
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, stratify=y_train, random_state=5)

In [30]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((42000, 784), (42000,), (18000, 784), (18000,))

(4) 학습 진행 <hr>

In [31]:
# (4-1) 분류 방법 : LogisticRegression
from sklearn.linear_model import LogisticRegression

In [32]:
# 스케일링을 안 해서 어제 30분 넘게 안 끝났구나!!!! 대박...
lr_model = LogisticRegression(max_iter=10000)
lr_model.fit(x_train, y_train)

LogisticRegression(max_iter=10000)

In [36]:
train_score = lr_model.score(x_train, y_train)
val_score = lr_model.score(x_val, y_val)
test_score = lr_model.score(x_test, y_test)

print(f'train_score : {train_score}, val_score : {val_score}, test_score : {test_score}')

train_score : 0.9429285714285714, val_score : 0.9173888888888889, test_score : 0.9238


In [34]:
# 모델 파라미터 
print(lr_model.n_iter_)     # 실제 학습 횟수

[721]


(4-2) 확률적 경사하강법(SGD) 기반 학습 <hr>
- 대량의 데이터를 학습 시 랜덤으로 데이터셋 1개 추출해서 학습 진행 => 최적화
- 장점 : 속도 빨라짐
- 단점 : 정확도 떨어짐
- 개선 : 일정 데이터를 묶어서 학습 후 최적화 진행 => 미니배치 
- 학습모델 선정 필요

In [40]:
# 모듈 로딩
# SGDClassifier => 분류 알고리즘을 SGD 방법으로 학습 진행
from sklearn.linear_model import SGDClassifier

In [44]:
# LogisticREgression 알고리즘으로 학습 진행 (loss='log_loss')
lr_sgd = SGDClassifier(loss='log_loss', random_state=5)
lr_sgd.fit(x_train, y_train)

SGDClassifier(loss='log_loss', random_state=5)

In [45]:
# 모델 파라미터 확인
print(lr_sgd.n_iter_)

24


In [46]:
# 성능 확인
train_score = lr_sgd.score(x_train, y_train)
val_score = lr_sgd.score(x_val, y_val)
test_score = lr_sgd.score(x_test, y_test)

print(f'train_score : {train_score}, val_score : {val_score}, test_score : {test_score}')

train_score : 0.9195714285714286, val_score : 0.909, test_score : 0.9127


In [52]:
## 추가 학습 전 모델의 계수, 절편 상태
print(f'[BEFORE] lr_sgd.coef_ : {lr_sgd.coef_}')      # (10, 784)
print(f'[BEFORE] lr_sgd.intercept_ : {lr_sgd.intercept_}')    # (10,)

[BEFORE] lr_sgd.coef_ : [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[BEFORE] lr_sgd.intercept_ : [-7.97799393 -0.99830779 -2.94178586 -4.36810167 -2.2825444   0.13435885
 -4.56454107 -0.85661241 -7.83566869 -4.44152253]


In [53]:
## 추가된 데이터에 대한 추가 학습 진행 => 기존 학습으로 최적화 된 상태 유지
import numpy as np

lr_sgd.partial_fit(x_val, y_val, np.unique(y_val))

SGDClassifier(loss='log_loss', random_state=5)

In [55]:
## 추가 학습 후 모델의 계수, 절편 상태
print(f'[AFTER] lr_sgd.coef_ : {lr_sgd.coef_}')      # (10, 784)
print(f'[AFTER] lr_sgd.intercept_ : {lr_sgd.intercept_}')    # (10,)

[AFTER] lr_sgd.coef_ : [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[AFTER] lr_sgd.intercept_ : [-7.89563585 -1.08504908 -2.9141212  -4.41080779 -2.26101996  0.15335754
 -4.51990011 -0.9033823  -7.74312773 -4.38734703]


(5) 학습 모델 성능 평가 <hr>

In [37]:
## 분류 모델 성능평가 => accuracy, precision, recall, f1-score => classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [39]:
# 클래스 비율
y_train.value_counts(), y_test.value_counts()

(class
 1    4719
 7    4385
 3    4292
 2    4171
 9    4164
 0    4146
 6    4143
 8    4096
 4    4089
 5    3795
 Name: count, dtype: int64,
 class
 1    1135
 2    1032
 7    1028
 3    1010
 9    1009
 4     982
 0     980
 8     974
 6     958
 5     892
 Name: count, dtype: int64)